# Project Title
### Data Engineering Capstone Project

#### Project Summary
In this project, a data warehouse for analytical purposes are created with an ETL pipeline. The database is designed as a star schema with transactions of purchase of fashion articles from H&M.

Following question could be answered:
- Which products group are most popular in year 2020?
- What is the distribution of user that buy the TShirt and also has active club member?


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Imports all dependency packages
import pandas as pd

### Step 1: Scope the Project and Gather Data

#### Scope

In this project, an ETL pipeline to provide the analytics team with a data model that can be used to recommend products to customers.

![Pipeline](./assets/pipeline.png)

Pipeline:
1. Gather data from kaggle
    For this project, manual download and upload into S3 will be used. In day to day operation, a streaming data pipeline, or directly accessing the data from the source DB, should be used. A pipeline that ingest the data source are recommended.
2. Save the csv data into staging area in redshift.
3. Save the partitioned image data (the images are already partitioned with the first three digist of article id)in S3.
4. Creating a star schema from the staging area in Redshift

Technology used:
- S3 : for saving source dataset
- Redshift : for staging and serves as datawarehouse
- Airflow : orchestration tool for ETL pipeline

In the future a spark cluster could be used for processing the data. Because the data we obtained are already cleaned, so we don't need to do any further processing and cleaning.

#### Describe and Gather Data

The dataset was obtained from [kagle competition](https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/data) dataset. The dataset are already cleaned. Gathering the data are done with the `kagle cli` with following command:
`kaggle competitions download -c h-and-m-personalized-fashion-recommendations`


The dataset contains the purchase history of the customers. Following files are included in the dataset:

- images/ - a folder of images corresponding to each article_id; images are partitioned with the first three digits of articles id.
- articles.csv - detailed metadata for each article_id available for purchase
- customers.csv - metadata for each customer_id in dataset
- transactions_train.csv - the purchases history each customer for each date.

##### Article Dataset

In [39]:
# Read in the data here
articles = pd.read_csv('dataset/full/articles.csv')
articles.head(20)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
5,110065011,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,12,Light Beige,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
6,111565001,111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
7,111565003,111565,20 den 1p Stockings,302,Socks,Socks & Tights,1010016,Solid,13,Beige,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
8,111586001,111586,Shape Up 30 den 1p Tights,273,Leggings/Tights,Garment Lower body,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Tights with built-in support to lift the botto...
9,111593001,111593,Support 40 den 1p Tights,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny tights that shape the tummy, thighs..."


In [14]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 25 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   article_id                    105542 non-null  int64 
 1   product_code                  105542 non-null  int64 
 2   prod_name                     105542 non-null  object
 3   product_type_no               105542 non-null  int64 
 4   product_type_name             105542 non-null  object
 5   product_group_name            105542 non-null  object
 6   graphical_appearance_no       105542 non-null  int64 
 7   graphical_appearance_name     105542 non-null  object
 8   colour_group_code             105542 non-null  int64 
 9   colour_group_name             105542 non-null  object
 10  perceived_colour_value_id     105542 non-null  int64 
 11  perceived_colour_value_name   105542 non-null  object
 12  perceived_colour_master_id    105542 non-null  int64 
 13 

In [15]:
print(f'Article dataset has {articles.shape[0]} rows and {articles.shape[1]} columns')

Article dataset has 105542 rows and 25 columns


##### Customers Dataset

In [16]:
# Read in the data here
customers = pd.read_csv('dataset/full/customers.csv')
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [17]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1371980 entries, 0 to 1371979
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   customer_id             1371980 non-null  object 
 1   FN                      476930 non-null   float64
 2   Active                  464404 non-null   float64
 3   club_member_status      1365918 non-null  object 
 4   fashion_news_frequency  1355971 non-null  object 
 5   age                     1356119 non-null  float64
 6   postal_code             1371980 non-null  object 
dtypes: float64(3), object(4)
memory usage: 73.3+ MB


In [18]:
print(f'Customer dataset has {customers.shape[0]} rows and {customers.shape[1]} columns')

Customer dataset has 1371980 rows and 7 columns


##### Transactions Dataset

In [21]:
transactions = pd.read_csv('dataset/full/transactions_train.csv')
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [22]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


In [23]:
print(f'Transactions dataset has {transactions.shape[0]} rows and {transactions.shape[1]} columns')

Transactions dataset has 31788324 rows and 5 columns


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.


##### Articles Dataset

In [24]:
# Check for missing values
articles.isnull().sum()

article_id                        0
product_code                      0
prod_name                         0
product_type_no                   0
product_type_name                 0
product_group_name                0
graphical_appearance_no           0
graphical_appearance_name         0
colour_group_code                 0
colour_group_name                 0
perceived_colour_value_id         0
perceived_colour_value_name       0
perceived_colour_master_id        0
perceived_colour_master_name      0
department_no                     0
department_name                   0
index_code                        0
index_name                        0
index_group_no                    0
index_group_name                  0
section_no                        0
section_name                      0
garment_group_no                  0
garment_group_name                0
detail_desc                     416
dtype: int64

=> there are 416 missing values for the column `details_desc` in the article dataset. This will be not a problem, since the `article id` is the most important information.

In [34]:
# check duplicates
articles.duplicated(subset='article_id').sum()

0

=> there are no duplicate `article_id` in the article dataset.

In [42]:
articles.head(10)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
5,110065011,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,12,Light Beige,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
6,111565001,111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
7,111565003,111565,20 den 1p Stockings,302,Socks,Socks & Tights,1010016,Solid,13,Beige,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
8,111586001,111586,Shape Up 30 den 1p Tights,273,Leggings/Tights,Garment Lower body,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,Tights with built-in support to lift the botto...
9,111593001,111593,Support 40 den 1p Tights,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny tights that shape the tummy, thighs..."


In [40]:
# check categorical data
articles.nunique()

article_id                      105542
product_code                     47224
prod_name                        45875
product_type_no                    132
product_type_name                  131
product_group_name                  19
graphical_appearance_no             30
graphical_appearance_name           30
colour_group_code                   50
colour_group_name                   50
perceived_colour_value_id            8
perceived_colour_value_name          8
perceived_colour_master_id          20
perceived_colour_master_name        20
department_no                      299
department_name                    250
index_code                          10
index_name                          10
index_group_no                       5
index_group_name                     5
section_no                          57
section_name                        56
garment_group_no                    21
garment_group_name                  21
detail_desc                      43404
dtype: int64

=> most of the feature are categorical data. => need to be normalized.
**NOTE** : Check if the data are unique! (UNIQUE in DB schema)

##### Customer Dataset

In [28]:
# check missing values in the customer dataset
customers.isnull().sum()

customer_id                    0
FN                        895050
Active                    907576
club_member_status          6062
fashion_news_frequency     16009
age                        15861
postal_code                    0
dtype: int64

=> there are no missing `customer_id in the customer dataset.

In [31]:
# check dupicates customer_id
customers.duplicated(subset='customer_id').sum()

0

=> there are no duplicate `customer_id` in the customer dataset.

In [43]:
customers.head(10)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
5,000064249685c11552da43ef22a5030f35a147f723d5b0...,NaN,NaN,NaN,NaN,NaN,2c29ae653a9282cce4151bd87643c907644e09541abc28...
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,NaN,NaN,ACTIVE,NONE,20.0,fe7b8e2b3fafb89ca90db17ffeeae0fd29b795d803f749...
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,1.0,1.0,ACTIVE,Regularly,32.0,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...
8,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...
9,00008469a21b50b3d147c97135e25b4201a8c58997f787...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...


In [44]:
# check categorical data
customers.nunique()

customer_id               1371980
FN                              1
Active                          1
club_member_status              3
fashion_news_frequency          4
age                            84
postal_code                352899
dtype: int64

=> there is no need to normalize the data

=> `FN` and `Active` could be converted into boolean.

##### Transactions Dataset

In [35]:
# check missing values in the transactions dataset
transactions.isnull().sum()

t_dat               0
customer_id         0
article_id          0
price               0
sales_channel_id    0
dtype: int64

=> there are no missing values in the transactions dataset.

In [37]:
# check duplicates customer_id and articles_id in the transactions
transactions.duplicated(subset=['customer_id', 'article_id']).sum()

4481885

=> there are no duplicate `customer_id` and `article_id` in the transactions dataset. Which means that the customer buy the same thing twice and will not be a problem for our pipeline

In [45]:
transactions.head(10)

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
5,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687001,0.016932,2
6,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221001,0.020322,2
7,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,688873012,0.030492,1
8,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,501323011,0.053373,1
9,2018-09-20,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,598859003,0.045746,2


In [46]:
transactions.nunique()

t_dat                   734
customer_id         1362281
article_id           104547
price                  9857
sales_channel_id          2
dtype: int64

#### Cleaning Steps
There is no need to clean the dataset, since the data already cleaned.

### Step 3: Define the Data Model

#### 3.1 Conceptual Data Model
![ERD of the starschema](./assets/star_schema.png)
The data model will be a star schema with `transactions` as the **FACT** table, and `customers` and `articles` as the **DIMENSION** tables. The `articles`table will have another metatable that improves the information of the `articles` table.

The `transactions` are used as **FACT** table, because it contains the most of the transaction data (movement data/events). The colom `id` or `code` or `no` are dropped, since they are not useful for our analysis. But the `articles` will have another column `has_image` that indicates whether the article has an image or not.

---

**Star Schema**
Star schema are chosen, because the query from the business analyst (requirement) are mostly related on the transactions of the customer. With the star schema, the query will be more efficient. The 
disadvantages will be that the data integrity is not guaranteed, but this could be checked with the data quality check.

Snowflake Schema is also a consideration, but the business analyst prefer faster approach to create a faster reporting and act based on it (such as send promotion email to the customer).

---

#### 3.2 Mapping Out Data Pipelines
The RAW data and images information are loaded in the staging area in `redshift`. From there, the data will be loaded into the fact and dimension tables.
The `articles` dataset will be only selected for the necessary information, the id of the metadata will be removed.

### Step 4: Run Pipelines to Model the Data
This step will be directly implemented in the code as Airflow Operator


#### 4.1 Data Quality Checks
Following data quality checks are performed:
- check if there are data in the `articles`, `customers` and `transactions`
- check if all datas in `staging_articles` are transformed into `articles`
- check if all datas in `customers` and `transactions` are unique.

#### 4.2 Query Examples
Following question could be answered:
- Which products group are most popular in year 2020?
  ```sql
  SELECT product_type_name,
         COUNT(*)
  FROM fact_transactions ft
         JOIN dim_articles da ON ft.article_id = da.article_id
  WHERE DATE_PART_YEAR(date_of_purchase) = 2020
  GROUP BY product_type_name
  ORDER BY COUNT(*) DESC;
  ```
- What is the distribution of user that buy the TShirt and also has active club member?
  ```sql
  SELECT is_active_club_member_status,
         COUNT(*)
  FROM fact_transactions ft
         JOIN dim_articles da ON ft.article_id = da.article_id
      JOIN dim_customers dc ON ft.customer_id = dc.customer_id
  WHERE product_type_name = 'T-shirt'
  GROUP BY is_active_club_member_status
  ```

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

Technology Stack:
- S3 are used because its capability to save big data and images (images in this project is around 35 GB)
- Redshift are used to stage the s3 into the staging area, because its fast and easy to access and a native service from AWS
- Airflow is used to run the ETL pipeline

Data update frequency:
For answering the given query, the data could be updated daily. To generate the report the analyst does not need the actual data.

For time crucial data, such as customer winback (if a customer want to delete their account), then a realtime data will be great, so the customer could get a realtime offer before deleting their account.

A real time data could be achieved by using pyspark streaming that read the stream and ingest the data into the staging area. Another "cron" task could be scheduled to ingest the data into the fact and dimension tables by appending into it.

Possible Scenario:
- **The data was increased by 100x.**
  => With the solution in this ETL pipeline, this should give no problem at all. The heavy task are given to the cloud (Redshift) and the calculation are based on sql, which run also on cloud. The airflow is just used as an orchestration tool for maintaining the data movement.
- **The data populates a dashboard that must be updated on a daily basis by 7am every day.**
  => Because of the time constraint, the data could be triggered early in the morning and this should be also not a problem at all. An SLA could be set for the data to be updated before 7am. If the SLA could not be achieved, then the redshift cluster could be scaled up to accommodate the data.
- **The database needed to be accessed by 100+ people.**
  => This will be no problem at all, since we are using the native internal table from Redshift. The query performance will be much better than the external table (reading from S3).
